<h1><center>Cover Letter Generator<center><h1>

---



In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec 18 01:20:02 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install "transformers[sentencepiece]==4.32.1" "optimum==1.12.0" "auto-gptq==0.4.2" "accelerate==0.22.0" "safetensors>=0.3.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: t

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import time

model_name_or_path = "TheBloke/Llama-2-13b-Chat-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)



# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

quantize_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

In [ ]:
start = time.time()

prompt = "Tell me about AI"
system_message = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''

response = pipe(prompt_template)
end = time.time()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
print(response[0]["generated_text"].split("[/INST]")[1])

  Hello! I'd be happy to help answer your questions about AI. Before we begin, I want to point out that the term "AI" can be somewhat misleading, as it implies that there is a single, monolithic technology that encompasses all aspects of artificial intelligence. In reality, AI is a broad field that includes many different approaches, techniques, and applications.

That being said, AI refers to the use of algorithms and machine learning techniques to perform tasks that typically require human intelligence, such as understanding natural language, recognizing images, making decisions, and solving problems. These technologies have been applied in a wide range of domains, including healthcare, finance, transportation, education, and entertainment.

There are several key concepts within the field of AI that are important to understand:

1. Machine learning: This is a subset of AI that involves training algorithms on large datasets in order to enable them to make predictions or take actions b

In [ ]:
print(f"Total time {(end - start)/60:.2f} min")

Total time 0.41 min


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/My Drive/CS505/CS505 Project/Database_prerow/Resume_data/Clear_Name_dataset.csv'
resume = pd.read_csv(file_path)

print(resume.head())

         JOB NAME                                             Resume  \
0  Accountant Job  ACCOUNTANT Professional Summary Results orient...   
1  Accountant Job  STAFF ACCOUNTANT Summary Flexible Accountant w...   
2  Accountant Job  STAFF ACCOUNTANT Summary Highly analytical and...   
3  Accountant Job  SENIOR ACCOUNTANT Summary A highly competent m...   
4  Accountant Job  SENIOR ACCOUNTANT Summary 11 years experience ...   

  First Name Last Name    Phone Number                           Email  \
0      Kayla   Pearson  (537) 736-5498  rebekahfitzpatrick@hotmail.com   
1    Timothy    Harmon  (395) 168-9140             othomas@hotmail.com   
2    Monique    Steele  (938) 695-0968            vpollard@hotmail.com   
3        Amy    Larson  (243) 045-8562              tjohnson@gmail.com   
4       Lisa   Coleman  (657) 918-2427           matthew12@hotmail.com   

                             LinkedIn Website  
0   https://www.linkedin.com/in/kayla-pearson  
1  https://www.linkedin.co

In [ ]:
selected_resume = resume[resume['JOB NAME'] == 'Data Science Job']
selected_resume.head()

,JOB NAME,Resume,First Name,Last Name,Phone Number,Email,LinkedIn Website
1426,Data Science Job,Skills Programming Languages Python pandas num...,Tristan,Clark,(851) 680-5921,marquezpatricia@gmail.com,https://www.linkedin.com/in/tristan-clark
1427,Data Science Job,Personal Skills Ability to quickly grasp techn...,Melanie,Anderson,(454) 084-4572,murillorickey@outlook.com,https://www.linkedin.com/in/melanie-anderson
1428,Data Science Job,Education Details B Tech Rayat and Bahra Insti...,Deborah,Li,(772) 994-4522,timothy12@hotmail.com,https://www.linkedin.com/in/deborah-li
1429,Data Science Job,Skills Python Tableau Data Visualization R Stu...,Ryan,Riley,(293) 136-0336,istevenson@outlook.com,https://www.linkedin.com/in/ryan-riley
1430,Data Science Job,SKILLS C Basics IOT Python MATLAB Data Science...,Robert,Collins,(201) 027-9915,dbecker@gmail.com,https://www.linkedin.com/in/robert-collins


In [ ]:
resume1 = selected_resume.iloc[3]
resume1

JOB NAME                                             Data Science Job
Resume              Skills Python Tableau Data Visualization R Stu...
First Name                                                       Ryan
Last Name                                                       Riley
Phone Number                                           (293) 136-0336
Email                                          istevenson@outlook.com
LinkedIn Website               https://www.linkedin.com/in/ryan-riley
Name: 1429, dtype: object

In [ ]:
file_path2 = '/content/drive/My Drive/CS505/CS505 Project/Database_prerow/Job_target_data/Job_targets.csv'
job = pd.read_csv(file_path2, encoding='ISO-8859-1')

print(job.head())

       job_id company_id                                              title  \
0  3757940104     553718                              Hearing Care Provider   
1  3757940025    2192142  Shipping & Receiving Associate 2nd shift (Beav...   
2  3757938019     474443                               Manager, Engineering   
3  3757938018   18213359                                               Cook   
4  3757937095     437225        Principal Cloud Security Architect (Remote)   

                                         description max_salary med_salary  \
0  Overview\n\nHearingLife is a national hearing ...        NaN       5250   
1  Metalcraft of Mayville\nMetalcraft of Mayville...        NaN        NaN   
2  \nThe TSUBAKI name is synonymous with excellen...        NaN        NaN   
3  descriptionTitle\n\n Looking for a great oppor...        NaN      22.27   
4  Job Summary\nAt iHerb, we are on a mission to ...     275834        NaN   

  min_salary pay_period formatted_work_type          loc

<ipython-input-19-ad1f46524305>:2: DtypeWarning: Columns (0,1,5,6,10,12,13,17,18,21,23,27,28,29,30,31,33,34,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  job = pd.read_csv(file_path2, encoding='ISO-8859-1')


In [ ]:
job.shape

(33571, 42)

In [ ]:
job = job.iloc[:, :28]
job.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped
0,3757940104,553718,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250,NaN,MONTHLY,Full-time,"Little River, SC",...,NaN,Entry level,NaN,1.70E+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101
1,3757940025,2192142,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",...,NaN,NaN,NaN,1.70E+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420
2,3757938019,474443,"Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",...,NaN,NaN,Bachelor's Degree in Mechanical Engineering pr...,1.70E+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644
3,3757938018,18213359,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,NaN,Entry level,NaN,1.70E+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461
4,3757937095,437225,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834,NaN,205956,YEARLY,Full-time,United States,...,NaN,Mid-Senior level,NaN,1.70E+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346


In [ ]:
selected_job = job[job.title == 'Data Scientist']
selected_job.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped
2396,3757733119,26945,Data Scientist,It's fun to work in a company where people tru...,208000,NaN,90000,YEARLY,Full-time,"California, United States",...,NaN,Entry level,NaN,1.70E+12,fractal.wd1.myworkdayjobs.com,0,FULL_TIME,USD,BASE_SALARY,1699080458
3022,3757722920,15721,Data Scientist,"At Extra Space Storage, what matters to you, m...",NaN,NaN,NaN,NaN,Full-time,"Salt Lake City, UT",...,NaN,Entry level,NaN,1.70E+12,extraspace.wd5.myworkdayjobs.com,0,FULL_TIME,NaN,NaN,1699072327
3277,3757709060,23099,Data Scientist,Become an essential member of NRG as a Data Sc...,65,NaN,55,HOURLY,Contract,"Houston, TX",...,NaN,Associate,NaN,1.70E+12,NaN,0,CONTRACT,USD,BASE_SALARY,1699052505
4026,3757495140,15226602,Data Scientist,*THIS ROLE IS NOT OPEN TO C2C OR H1B. SUCCESSF...,NaN,NaN,NaN,NaN,Contract,United States,...,NaN,Mid-Senior level,NaN,1.70E+12,NaN,0,CONTRACT,NaN,NaN,1699063864
4551,3757490630,80753330,Data Scientist,Job Title\n\nData Scientist\n\nJob Description...,130000,NaN,98000,YEARLY,Full-time,United States,...,NaN,Entry level,NaN,1.70E+12,meredith.wd5.myworkdayjobs.com,0,FULL_TIME,USD,BASE_SALARY,1699090221


In [ ]:
job1 = selected_job.iloc[2]
job1

job_id                                                               3757709060
company_id                                                                23099
title                                                            Data Scientist
description                   Become an essential member of NRG as a Data Sc...
max_salary                                                                   65
med_salary                                                                  NaN
min_salary                                                                   55
pay_period                                                               HOURLY
formatted_work_type                                                    Contract
location                                                            Houston, TX
applies                                                                      10
original_listed_time                                                   1.70E+12
remote_allowed                          

In [ ]:
def generate_cv(first_name, last_name, content, position, jd):

  prompt = f"Create a cover letter for {first_name} {last_name} based on the applicant's resume: /n{content}/n Tailor the resume to fit the position {position} of this job description: /n{jd}/n"
  system_message = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
  prompt_template=f'''[INST] <<SYS>>
  {system_message}
  <</SYS>>

  {prompt} [/INST]'''

  response = pipe(prompt_template)
  return response[0]["generated_text"].split("[/INST]")[1]

In [ ]:
generate_cv(resume1['First Name'], resume1['Last Name'], resume1['Resume'], job1['title'], job1['description'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


"  Dear Hiring Manager,\n\nI am writing to apply for the position of Data Scientist at NRG, as advertised on [website]. With over 3 years of experience in statistical modeling and quantitative analysis, I am confident that my skills and expertise align with the requirements of this role.\n\nAs a certified data scientist with a Bachelor's degree in Computer Science and a strong background in machine learning, statistics, and data visualization, I possess a unique combination of technical and domain expertise that makes me an ideal candidate for this position. My experience includes:\n\n* 13 months of experience in Python programming, including data analysis and machine learning tasks\n* 24 months of experience in data science, including data visualization and tableau\n* Strong understanding of graduate-level multivariate statistical techniques and sampling methods, including but not limited to multivariate regression, ANOVA, factor analysis, cluster analysis, and principal components an

In [ ]:
def generate_cv2(first_name, last_name, content, position, jd):
  start = time.time()
  prompt = f"Create a cover letter for {first_name} {last_name} based on the applicant's resume: /n{content}/n/n Tailor the resume to fit the position {position} of this job description: /n{jd}/n/n cover letter that is professional but concise with 2 paragraphs"
  system_message = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
  prompt_template=f'''[INST] <<SYS>>
  {system_message}
  <</SYS>>

  {prompt} [/INST]'''

  response = pipe(prompt_template)
  end = time.time()
  return response[0]["generated_text"].split("[/INST]")[1]

In [ ]:
generate_cv2(resume1['First Name'], resume1['Last Name'], resume1['Resume'], job1['title'], job1['description'])

'  Dear Hiring Manager,\n\nI am writing to express my strong interest in the Data Scientist position at NRG. As a highly motivated and experienced data scientist with a background in computer science, economics, and mathematics, I am confident that I can bring significant value to your organization.\n\nWith over three years of experience in statistical modeling and quantitative analysis, I possess a strong foundation in multivariate statistical techniques and sampling methods, including multivariate regression, ANOVA, factor analysis, cluster analysis, and principal components analysis. My expertise in Python and SQL allows me to efficiently manipulate and analyze large datasets, and I have a proven track record of developing and implementing machine learning algorithms to solve complex business problems. Additionally, I have extensive experience with data visualization tools such as Tableau, allowing me to effectively communicate complex insights to both technical and non-technical au

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import subprocess
subprocess.run(["pip", "install", "streamlit"])

CompletedProcess(args=['pip', 'install', 'streamlit'], returncode=0)

In [ ]:
!pip install -q streamlit

In [ ]:
%%writefile cl.py
import streamlit as st

st.title('Cover Letter Generator')
st.markdown("This app will generate a cover letter for you based on your resume.")

answer_container = st.container()
with answer_container:
    col1, col2, col3 = st.columns([1.5, 1.5, 1])

    response = generate_cv2(resume1['First Name'], resume1['Last Name'], resume1['Resume'], job1['title'], job1['description'])

    with col1:
      st.subheader("Candidate")
      st.write(f'Name: {resume1["First Name"]}, {resume1["Last Name"]}')
      st.write("###")
      st.write(resume1['Resume'])
    with col2:
      st.subheader("Generated Cover Letter")
      st.write(response)

    with col3:
      st.subheader("Evaluation Metrics")

Writing cl.py


In [ ]:
# import locale
# import os

# os.environ['LC_ALL'] = 'C.UTF-8'
# os.environ['LANG'] = 'C.UTF-8'
# locale.setlocale(locale.LC_ALL, 'C.UTF-8')

In [ ]:
# !curl -sL https://deb.nodesource.com/setup_14.x | sudo -E bash -
# !sudo apt-get install -y nodejs
# # !node --version
# !npm --18.17.0

In [ ]:
!npm audit fix

npm ERR! code EAUDITNOPJSON
npm ERR! audit No package.json found: Cannot audit a project without a package.json

npm ERR! A complete log of this run can be found in:
npm ERR!     /root/.npm/_logs/2023-12-18T01_22_57_848Z-debug.log


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.279s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


   ╭────────────────────────────────────────────────────────────────╮
   │                                                                │
   │      New major version of npm available! 6.14.8 → 10.2.5       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v10.2.5   │
   │               Run npm install -g npm to update!                │
   │ 

In [ ]:
!streamlit run cl.py &>/content/logs.txt & curl ipv4.icanhazip.com

34.91.131.206


In [ ]:
!npx localtunnel --port 4000

npx: installed 22 in 1.158s
your url is: https://cold-lands-sin.loca.lt
^C


In [ ]:
from IPython.display import display
from ipywidgets import widgets, Layout, Box, GridspecLayout

In [ ]:
# Input Widgets
first_name_input = widgets.Text(description='First Name:')
last_name_input = widgets.Text(description='Last Name:')
resume_input = widgets.Textarea(description='Resume:')
job_title_input = widgets.Text(description='Job Title:')
job_description_input = widgets.Textarea(description='Job Description:')

# Output Widgets
candidate_output = widgets.Textarea(description='Candidate:', disabled=True)
cover_letter_output = widgets.Textarea(description='Cover Letter:', disabled=True)
evaluation_metrics_output = widgets.Textarea(description='Evaluation Metrics:', disabled=True)

In [ ]:
def generate_cover_letter(b):
    # Use the input values to generate a cover letter
    # For example, using a placeholder function generate_cv2
    response = generate_cv2(first_name_input.value, last_name_input.value, resume_input.value, job_title_input.value, job_description_input.value)

    # Update Output Widgets
    candidate_output.value = f"Name: {first_name_input.value}, {last_name_input.value}\nResume:\n{resume_input.value}"
    cover_letter_output.value = response
    # Update evaluation_metrics_output as needed

In [ ]:
generate_button = widgets.Button(description="Generate Cover Letter")
generate_button.on_click(generate_cover_letter)

In [ ]:
# Create a layout
layout = GridspecLayout(3, 3)
layout[0, 0] = first_name_input
layout[1, 0] = last_name_input
layout[2, 0] = generate_button
layout[:, 1] = resume_input
layout[:, 2] = job_description_input

# Display the layout
display(layout)

GridspecLayout(children=(Text(value='', description='First Name:', layout=Layout(grid_area='widget001')), Text…

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
display(candidate_output, cover_letter_output, evaluation_metrics_output)

Textarea(value='Name: , \nResume:\n', description='Candidate:', disabled=True)

Textarea(value='  Dear Hiring Manager,\n\nI am writing to express my strong interest in the position of [Posit…

Textarea(value='', description='Evaluation Metrics:', disabled=True)